In [89]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [90]:
train_labeled = pd.read_csv('data/base_files/labled_train_data.csv', index_col=0, sep='\t', comment='#')
comments_labeled = pd.read_csv('data/base_files/labled_train_comments.csv', index_col=0, sep='\t', comment='#')
tracks_labeled = pd.read_csv('data/labled_train_tracks_speed.csv', index_col=0, sep=',', comment='#')

train_unlabeled = pd.read_csv('data/base_files/unlabled_train_data.csv', index_col=0, sep='\t', comment='#')
comments_unlabeled = pd.read_csv('data/base_files/unlabled_train_comments.csv', index_col=0, sep='\t', comment='#')
tracks_unlabeled = pd.read_csv('data/base_files/unlabled_train_tracks.csv', index_col=0, sep='\t', comment='#')

each order is different batch

In [166]:
# 1.
# train_data =  np.delete(tracks_labeled.values.tolist(), np.s_[0], 1) # delete index from each subarray
train_data = np.array([tracks_labeled[tracks_labeled['order_id']==order].values for order in tracks_labeled['order_id'].unique()[:5]])
train_labels = []
for order_batch in train_data:
    is_aggressive = order_batch[0][-1]  
    train_labels.append(is_aggressive)
train_labels = np.array(train_labels)
# train_data = np.concatenate(train_data,axis = 0)


<ipython-input-166-b043595f02d2>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_data = np.array([tracks_labeled[tracks_labeled['order_id']==order].values for order in tracks_labeled['order_id'].unique()[:5]])


In [164]:
train_data

array([[0, 'b76545fa3cc14acd6a69ac13c1edac33', '2021-02-09 21:43:41',
        ..., '001662da857b5a39bb402aacf3145f86', nan, 0.0],
       [1, 'b76545fa3cc14acd6a69ac13c1edac33', '2021-02-09 21:44:40',
        ..., '001662da857b5a39bb402aacf3145f86', 5.906440677966101, 0.0],
       [2, 'b76545fa3cc14acd6a69ac13c1edac33', '2021-02-09 21:45:00',
        ..., '001662da857b5a39bb402aacf3145f86', 15.696, 0.0],
       ...,
       [356, '8f5c5d3323bd8d9b21eae9d12224700b', '2021-03-05 20:10:38',
        ..., '002946d95ca1bc6954858b63c202bbb1', 9.782608695652174, 0.0],
       [357, '8f5c5d3323bd8d9b21eae9d12224700b', '2021-03-05 20:11:01',
        ..., '002946d95ca1bc6954858b63c202bbb1', 42.90260869565217, 0.0],
       [358, '8f5c5d3323bd8d9b21eae9d12224700b', '2021-03-05 20:11:23',
        ..., '002946d95ca1bc6954858b63c202bbb1', 14.514545454545456, 0.0]],
      dtype=object)

In [158]:
def preprocess_train(arr):
    # delete order id, index, driver id, aggressive label
    result = np.delete(arr, [0,1,3,4,5,7], 1)
    return result
X = preprocess_train(train_data)


In [160]:
from sklearn.model_selection import KFold  
from sklearn.pipeline import make_pipeline
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from minirocket.code.minirocket_multivariate_variable import fit, transform
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifierCV

X_train, X_test, y_train, y_test = train_test_split(X, train_labels, test_size=0.33, random_state=42)
parameters = fit(X_train, y_train)
X_train_transform = transform(X_train, y_train, parameters)

classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10), normalize = True)
classifier.fit(X_train_transform, y_train)



ValueError: Found input variables with inconsistent numbers of samples: [225, 5]